In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model
import os
import keras
import math
import numpy as np

# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
#top_model_weights_path = '../model/bottleneck_fc_model.h5'
top_model_weights_path = '../model/bottleneck_fc_model_inceptionv3_500_val_acc_4791.h5'
#top_model_weights_path = '../model/fc_model.h5'
img_width, img_height = 500, 500

#full
#train_data_dir = '../data/Pandora18K_train_val_test_split/train'
#validation_data_dir = '../data/Pandora18K_train_val_test_split/val'
#test_data_dir = '../data/Pandora18K_train_val_test_split/test'
#nb_train_samples = 14313
#nb_validation_samples = 1772
#nb_test_samples = 1791
#total_num_classes = 18

#small
train_data_dir = '../data/Pandora18K_small_train_val_test_split/train'
validation_data_dir = '../data/Pandora18K_small_train_val_test_split/val'
test_data_dir = '../data/Pandora18K_small_train_val_test_split/test'
nb_train_samples = 1462
nb_validation_samples = 167
nb_test_samples = 171
total_num_classes = 18

#verysmall
#train_data_dir = '../data/verysmall/train'
#validation_data_dir = '../data/verysmall/val'
#nb_train_samples = 144 #157
#nb_validation_samples = 16 #18
#total_num_classes = 2

epochs = 5
batch_size = 32

Using TensorFlow backend.


In [2]:

# build the VGG16 network
base_model = applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(img_width,img_height,3))
#base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dropout(0.6))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(.6))
top_model.add(Dense(18, activation='softmax'))

top_model.load_weights(top_model_weights_path)

model = Model(input= base_model.input, output= top_model(base_model.output))

Model loaded.


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("se..., inputs=Tensor("in...)`


In [3]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale= 1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

valid_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen  = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

if False:
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=1e-5, momentum=0.9),
                  metrics=['accuracy'])
    _, sanity_acc = model.evaluate_generator(generator = validation_generator, steps = nb_validation_samples // batch_size)
    print("sanity_acc = " + str(sanity_acc))


Found 1462 images belonging to 18 classes.
Found 167 images belonging to 18 classes.
Found 171 images belonging to 18 classes.


In [4]:
pretrained_model = "part_3_weights"
weight_dir = "../model/%s"%pretrained_model
if not os.path.exists(weight_dir):
    os.makedirs(weight_dir)

best_weight_path = "../model/best.h5"
best_acc = 0
best_lr = 0
best_momentum = 0
best_frozen = 0
num_parameter_sets = 2
    
for i in range(num_parameter_sets):
    print("\n")
    
    lr = 10**np.random.uniform(low=-7, high=-3)
    momentum = np.random.uniform(low = 0.7, high = 0.9)
    num_frozen = np.random.randint(300, high=312) #312 total
    print("lr = " + str(lr))
    print("momentum = " + str(momentum))
    print("number of frozen layers = " + str(num_frozen))
    
    # build the VGG16 network
    base_model = applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(img_height,img_width,3))

    # build a classifier model to put on top of the convolutional model
    top_model = Sequential()
    top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    top_model.add(Dropout(0.6))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(.6))
    top_model.add(Dense(18, activation='softmax'))

    top_model.load_weights(top_model_weights_path)

    model = Model(input= base_model.input, output= top_model(base_model.output))
    
    for layer in model.layers:
        layer.trainable = True
    
    for layer in model.layers[:num_frozen]:
        layer.trainable = False

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=lr),
                  momentum=momentum,
                  metrics=['accuracy'])
    
    param_names = "lr_" + str(lr)
    weight_path = os.path.join(weight_dir, param_names + "_best_weights%s.h5"%pretrained_model)
    checkpointer = keras.callbacks.ModelCheckpoint(filepath=weight_path, verbose=1,monitor='val_acc', save_best_only=True, save_weights_only=True)
    stopper = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=1, verbose=0, mode='auto')

    model.fit_generator(
        train_generator,
        steps_per_epoch=(nb_train_samples // batch_size)+1,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=(nb_validation_samples // batch_size)+1,
        callbacks=[checkpointer, stopper])
    
    model.load_weights(weight_path)
    new_loss, new_acc = model.evaluate_generator(generator = validation_generator, steps = (nb_validation_samples // batch_size)+1)
    print("new_acc = "+ str(new_acc))
    print("new_loss = "+ str(new_loss))
    
    if new_acc > best_acc:
        model.load_weights(weight_path)
        model.save_weights(best_weight_path, overwrite = True)
        best_acc = new_acc
        best_lr = lr
        best_momentum = momentum
        best_frozen = num_frozen
        
    os.remove(weight_path)
    
    print("best val acc so far = " + str(best_acc))
    

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:36: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("se..., inputs=Tensor("in...)`
/home/eraserwars/env_keras_tutorial/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:2250: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))




lr = 2.64282138957e-06
momentum = 0.882109323846
number of frozen layers = 308
Epoch 1/5
46/46 [==============================] - 91s - loss: 5.7305 - acc: 0.1582 - val_loss: 0.8682 - val_acc: 0.7665
Epoch 2/5
46/46 [==============================] - 75s - loss: 5.5952 - acc: 0.1460 - val_loss: 1.2150 - val_acc: 0.6347
Epoch 3/5
46/46 [==============================] - 75s - loss: 5.4205 - acc: 0.1696 - val_loss: 1.7337 - val_acc: 0.5389
new_acc = 0.766467066939
new_loss = 0.86823830526
best val acc so far = 0.766467066939


lr = 0.000741818684843
momentum = 0.833928268928
number of frozen layers = 310
Epoch 1/5
46/46 [==============================] - 90s - loss: 3.5547 - acc: 0.1934 - val_loss: 1.0465 - val_acc: 0.7305
Epoch 2/5
46/46 [==============================] - 75s - loss: 2.6227 - acc: 0.2126 - val_loss: 1.2877 - val_acc: 0.6347
Epoch 3/5
46/46 [==============================] - 75s - loss: 2.4918 - acc: 0.1990 - val_loss: 1.4512 - val_acc: 0.5868
new_acc = 0.730538923226


In [6]:
model.load_weights(best_weight_path)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=best_lr),
              momentum=best_momentum,
              metrics=['accuracy'])

test_loss, test_acc = model.evaluate_generator(generator = test_generator, steps = (nb_test_samples // batch_size)+1)
print("best test loss = " + str(test_loss))
print("best test acc = " + str(test_acc))

best test loss = 0.995896580624
best test acc = 0.66666666597
